Importing packages

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from selenium.webdriver.common.by import By

import time
#import langid

Initialisation

In [2]:
profile = FirefoxProfile("/Users/Alexandre/Library/Application Support/Firefox/Profiles/67y53u60.default-1491160038966/")
driver = webdriver.Firefox(profile)
del profile

Get players grades for 2016/2017 season

In [3]:
columns = ['Journee','Team','Adversaire','H/A','Player','Note']
mpg_df = pd.DataFrame(columns=columns)
del columns


#Specify the url
url_root = 'https://www.monpetitgazon.com/championships/1/match/'

#first match of the season
first_match = 853139

for j in range(1,2): 
    print 'Journée', j
    for m in range(1,11):
        url_j = first_match+(j-1)*10+(m-1)
        url = url_root + str(url_j)
        
        driver.get(url)
        home_club_name = str(driver.find_element_by_class_name('index__homeTeamClubName___21dga').text)
        #print repr(home_club_name)
        away_club_name = str(driver.find_element_by_class_name('index__awayTeamClubName___2U5C2').text)
        #print repr(away_club_name)
        print 'Extracting players grade for ', home_club_name, ' - ', away_club_name
        home = driver.find_element_by_class_name('index__pitchHome___1qSWd')
        away = driver.find_element_by_class_name('index__pitchAway___2sAoj')
        home_players = home.find_elements_by_class_name("index__player___2pAc8")
        away_players = away.find_elements_by_class_name("index__player___2pAc8")
        
        #print 'iterate throug home'
        for i, val in enumerate(home_players):
            data_player = {}
            data_player['Journee'] = j
            data_player['Team'] = home_club_name
            data_player['Adversaire'] = away_club_name
            data_player['H/A'] = 'H'
            rank_player = val.text
            #print repr(rank_player)
            split = rank_player.split('\n')
            #print split
            data_player['Player'] = split[1].encode('utf-8')
            data_player['Note'] = split[0].encode('utf-8')
            #print data_player
            #print type(data_player)
            mpg_df = mpg_df.append(data_player, ignore_index=True, verify_integrity=False)
            #print '\n'
        #print mpg_df    

        #print 'Iterate throug away'
        for i, val in enumerate(away_players):
            data_player = {}
            data_player['Journee'] = j
            data_player['Team'] = away_club_name
            data_player['Adversaire'] = home_club_name
            data_player['H/A'] = 'A'
            rank_player = val.text
            #print repr(rank_player)
            split = rank_player.split('\n')
            #print split
            data_player['Player'] = split[1].encode('utf-8')
            data_player['Note'] = split[0].encode('utf-8')
            #print data_player
            #print type(data_player)
            mpg_df = mpg_df.append(data_player, ignore_index=True, verify_integrity=False)
            #sprint '\n'
        #print mpg_df
    print '\n'

del j, m, i, val
del url_root, first_match, url_j, url
del home_club_name, away_club_name, home, away, home_players, away_players
del data_player, rank_player, split
driver.quit()
del driver
print 'Data scrapped'

Journée 1
Extracting players grade for  Nancy  -  Lyon
Extracting players grade for  Bordeaux  -  Saint Etienne
Extracting players grade for  Montpellier  -  Angers
Extracting players grade for  Nice  -  Rennes
Extracting players grade for  Marseille  -  Toulouse
Extracting players grade for  Bastia  -  Paris
Extracting players grade for  Dijon  -  Nantes
Extracting players grade for  Monaco  -  Guingamp
Extracting players grade for  Caen  -  Lorient
Extracting players grade for  Metz  -  Lille


Data scrapped


In [9]:
mpg_df.to_csv('player_data.csv',index = False)


Get match results for 2016/2017 games

In [10]:
profile = FirefoxProfile("/Users/Alexandre/Library/Application Support/Firefox/Profiles/67y53u60.default-1491160038966/")
driver = webdriver.Firefox(profile)
del profile

In [11]:
columns = ['Journee','Team','Adversaire','H/A','resultat','p.','c.']
resultat_df = pd.DataFrame(columns=columns)
del columns


#Specify the url
url_root = 'http://www.lfp.fr/ligue1/calendrier_resultat#sai=100&jour='
#print url_root

#first match of the season

for j in range(1,39): 
    print 'Journée', j
    url = url_root + str(j)
    print url
    driver.get(url)
    time.sleep(10)

    
    journee = driver.find_element_by_id('tableaux_rencontres')
    days = journee.find_elements(By.TAG_NAME, "tbody") 
    print 'nombre de jours de match', len(days)
    for d, day in enumerate(days):
        print '   day', d
        trs = day.find_elements(By.TAG_NAME, "tr")
        print '   nombre de match ce day', len(trs)
        for m, match in enumerate(trs):
            print '      match', m
            domicile = str(match.find_element_by_class_name('domicile').text.encode('utf-8'))
            print '         domicile', domicile
            exterieur = str(match.find_element_by_class_name('exterieur').text.encode('utf-8'))
            print '         exterieur', exterieur
            stats = str(match.find_element_by_class_name('stats').text)
            print '         stats', stats
            if not stats:
                continue
            else:
                score = stats.split(' - ')
            #print score

            # Equipe domicile
            data_match = {}
            data_match['Journee'] = j
            data_match['Team'] = domicile
            data_match['Adversaire'] = exterieur
            data_match['H/A'] = 'H'
            data_match['resultat'] = stats
            data_match['p.'] = score[0]
            data_match['c.'] = score[1]
            resultat_df = resultat_df.append(data_match, ignore_index=True, verify_integrity=False)

            # Equipe exterieur
            data_match = {}
            data_match['Journee'] = j
            data_match['Team'] = exterieur
            data_match['Adversaire'] = domicile
            data_match['H/A'] = 'A'
            data_match['resultat'] = stats
            data_match['p.'] = score[1]
            data_match['c.'] = score[0]
            resultat_df = resultat_df.append(data_match, ignore_index=True, verify_integrity=False)
    print '\n'
    
del j,journee, days, d, day,trs, m, match
del url_root, url
del domicile, exterieur, stats, score, data_match
driver.quit()
del driver
print 'Data scrapped'
resultat_df

Journée 1
http://www.lfp.fr/ligue1/calendrier_resultat#sai=100&jour=1
nombre de jours de match 3
   day 0
   nombre de match ce day 2
      match 0
         domicile SC Bastia
         exterieur Paris Saint-Germain
         stats 0 - 1
      match 1
         domicile AS Monaco
         exterieur EA Guingamp
         stats 2 - 2
   day 1
   nombre de match ce day 5
      match 0
         domicile Girondins de Bordeaux
         exterieur AS Saint-Etienne
         stats 3 - 2
      match 1
         domicile SM Caen
         exterieur FC Lorient
         stats 3 - 2
      match 2
         domicile Dijon FCO
         exterieur FC Nantes
         stats 0 - 1
      match 3
         domicile FC Metz
         exterieur LOSC
         stats 3 - 2
      match 4
         domicile Montpellier Hérault SC
         exterieur Angers SCO
         stats 1 - 0
   day 2
   nombre de match ce day 3
      match 0
         domicile AS Nancy Lorraine
         exterieur Olympique Lyonnais
         stats 0 - 3
  

,Journee,Team,Adversaire,H/A,resultat,p.,c.
0,1.0,SC Bastia,Paris Saint-Germain,H,0 - 1,0,1
1,1.0,Paris Saint-Germain,SC Bastia,A,0 - 1,1,0
2,1.0,AS Monaco,EA Guingamp,H,2 - 2,2,2
3,1.0,EA Guingamp,AS Monaco,A,2 - 2,2,2
4,1.0,Girondins de Bordeaux,AS Saint-Etienne,H,3 - 2,3,2
5,1.0,AS Saint-Etienne,Girondins de Bordeaux,A,3 - 2,2,3
6,1.0,SM Caen,FC Lorient,H,3 - 2,3,2
7,1.0,FC Lorient,SM Caen,A,3 - 2,2,3
8,1.0,Dijon FCO,FC Nantes,H,0 - 1,0,1
9,1.0,FC Nantes,Dijon FCO,A,0 - 1,1,0


In [12]:
resultat_df.to_csv('game_data.csv',index = False)


In [104]:
whos

Variable         Type                 Data/Info
-----------------------------------------------
BeautifulSoup    type                 <class 'bs4.BeautifulSoup'>
By               type                 <class 'selenium.webdriver.common.by.By'>
FirefoxProfile   type                 <class 'selenium.webdrive<...>_profile.FirefoxProfile'>
days             list                 n=1
i                int                  0
journee          FirefoxWebElement    <selenium.webdriver.firef<...>8644-b48a-8a2048aad351")>
matches          FirefoxWebElement    <selenium.webdriver.firef<...>d648-b0e1-51857c7e545e")>
mpg_df           DataFrame            Empty DataFrame\nColumns:<...> Player, Note]\nIndex: []
np               module               <module 'numpy' from '/Us<...>ages/numpy/__init__.pyc'>
pd               module               <module 'pandas' from '/U<...>ges/pandas/__init__.pyc'>
resultat_df      DataFrame                 Journee             <...>n\n[460 rows x 7 columns]
time             m

Other code - example

In [ ]:
def transformUrl(word):
    if word[0:8] == 'https://':
        word = word[8:]
    if word[0:7] == 'http://':
        word = word[7:]
    if word[0:4] == 'www.':
        word = word[4:]
    if '/' in word:
        word2 = word.split('/')
        word=word2[0]
    if '?' in word:
        word2 = word.split('?')
        word=word2[0]        
    return word


def predLang(url, txt):
    url = transformUrl(url)
    history = pandas.read_csv('langResponses.csv')
    for index, row in history.iterrows():
            if row['Website'] == url:
                return str(row['Language'])
            
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}
    dfCsv = pandas.DataFrame()
    dfCsv['Website'] = [url]

    url = 'http://' + url
    req = urllib2.Request(url, headers=hdr)
    try:
        resp = urllib2.urlopen(req)
        html = resp.read()
        soup = BeautifulSoup(html, "lxml")
    # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

    # get text
        text = soup.get_text()

    # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        if txt:
            print text
        if len(text) <10:
            dfCsv['Language'] = 'undefined'
            dfCsv.to_csv('langResponses.csv', mode='a', header=False, index=False)
            return 'undefined'
        else:
            if '403 Forbidden' in text or 'Access Denied' in text or 'Request unsuccessful' in text or 'Please enable JavaScript to view the page content' in text:
                dfCsv['Language'] = 'undefined'
                dfCsv.to_csv('langResponses.csv', mode='a', header=False, index=False)
                return 'undefined'
            elif 'has banned your access based on your browser\'s signature' in text:
                dfCsv['Language'] = 'undefined'
                dfCsv.to_csv('langResponses.csv', mode='a', header=False, index=False)
                return 'undefined'
            else:
                dfCsv['Language'] = langid.classify(text)[0]
                dfCsv.to_csv('langResponses.csv', mode='a', header=False, index=False)
                return langid.classify(text)[0]

    except:
        dfCsv['Language'] = 'error'
        dfCsv.to_csv('langResponses.csv', mode='a', header=False, index=False)
        return 'error'